In [ ]:
!pip install PyTDC

In [1]:
from tdc.single_pred.mpc import MPC
data = MPC(name = "https://raw.githubusercontent.com/bidd-group/MPCD/main/dataset/ADMET/DeepDelta_benchmark/Caco2.csv") # url from the source github repo https://github.com/bidd-group/MPCD/tree/main/dataset
# example url: https://raw.githubusercontent.com/bidd-group/MPCD/main/dataset/ADMET/DeepDelta_benchmark/Caco2.csv
split = data.get_split()

100%|██████████| 910/910 [00:03<00:00, 241.78it/s]


In [2]:
train_data = split['train']
test_data = split['test']

In [3]:
X_train = train_data['SMILES']
y_train = train_data['Y']
X_test = test_data['SMILES']
y_test = test_data['Y']

In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles_to_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)

X_train_fp = [smiles_to_fp(s) for s in X_train]
X_test_fp = [smiles_to_fp(s) for s in X_test]


In [ ]:
X_train_fp

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = RandomForestRegressor(n_estimators=23, random_state=42)
model.fit(X_train_fp, y_train)

y_pred = model.predict(X_test_fp)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.2961497651571661
